# Comparing Pancakeswap GetReserves() Speed

This notebook compares the speed of `GetReserves()` between bloXroute cloud API and standard fullnodes.

Start a fullnode without a local gateway, and run [fullnode_pair_reserve](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/pair_reserve/fullnode_pair_reserve.go) and [bloxroute_pair_reserve](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/bloxroute/pair_reserve/bloxroute_pair_reserve.go) commands to collect data.

Note: Make sure the BSC fullnode is fully synced before this benchmark, type `eth.syncing` in geth console and make sure it is `faluse`, which means the fullnode is synced.

Hardware:

* A machine with 128 cores, 256GB memory in Fremont IDC, running a BSC fullnode with a local bloXroute gateway
* A `m5zn.3xlarge` instance in AWS Virginia regon, running a BSC fullnode with a local bloXroute gateway

There are 5 ways to get pair reserves:

1. Query the local fullnode for each pair periodically. 

    * Code [fullnode_pair_reserve](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/pair_reserve/fullnode_pair_reserve.go)
    * Data files: `fullnode-pair-reserve-fremont.json.xz`, `fullnode-pair-reserve-virginia.json.xz`.

1. Query the local fullnode for all pairs(via the BulkReader smart contract) periodically.

    * Code [fullnode_pair_reserve_bulk](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/pair_reserve_v2/fullnode_pair_reserve_bulk.go)
    * Data files: `fullnode-pair-reserve-bulk-fremont.json.xz`, `fullnode-pair-reserve-bulk-virginia.json.xz`.

1. Query the local fullnode for all pairs(via the BulkReader smart contract) when a new block arrives.

    * Code [fullnode_pair_reserve_bulk_head](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/pair_reserve_v3/fullnode_pair_reserve_bulk_head.go)
    * Data files: `fullnode-pair-reserve-bulk-header-fremont.json.xz`, `fullnode-pair-reserve-bulk-header-virginia.json.xz`.

1. Subscribe the `ethOnBlock` stream from a BloXroute local gateway and call the the BulkReader smart contract.

    * Code [bloxroute_pair_reserve](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/bloxroute/pair_reserve/bloxroute_pair_reserve.go)
    * Data files: `bloxroute-pair-reserve-gateway-fremont.json.xz`, `bloxroute-pair-reserve-gateway-virginia.json.xz`.

1. Subscribe the `ethOnBlock` stream from BloXroute cloud and call the the BulkReader smart contract.

    * Code [bloxroute_pair_reserve](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/bloxroute/pair_reserve/bloxroute_pair_reserve.go)
    * Ddata files: `bloxroute-pair-reserve-cloud-fremont.json.xz`, `bloxroute-pair-reserve-cloud-virginia.json.xz`.

In [1]:
import gzip
import json
from json.decoder import JSONDecodeError
import lzma
import pandas as pd
from typing import Dict

In [2]:
pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

In [3]:
def read_timestamp(file: str) -> Dict[int, int]:
    result: Dict[str, int] = {}  # hash -> received_at
    f_in = None
    if file.endswith('.json'):
        f_in = open(file, "rt")
    elif file.endswith('.json.gz'):
        f_in = gzip.open(file, "rt")
    elif file.endswith('.json.xz'):
        f_in = lzma.open(file, "rt")
    else:
        raise ValueError(file)
    with f_in:
        for line in f_in:
            try:
                obj = json.loads(line)
                hash_code = hash(obj['pair'] + obj['reserve0'] + obj['reserve1'] + str(obj['block_number']) + str(obj['block_timestamp_last']))
                result[hash_code] = obj['received_at']
            except JSONDecodeError as ex:
                pass # ignore malformed lines
    return result

In [4]:
def compare_timestamp(file1: str, file2: str) -> pd.DataFrame:
    dict1 = read_timestamp(file1)
    dict2 = read_timestamp(file2)
    diff = {}
    for hash_code, received_at in dict1.items():
        if hash_code in dict2:
            diff[hash_code] = received_at-dict2[hash_code]
    
    df = pd.DataFrame(diff.items(), columns=['hash_code', 'gap'])
    # remove outliers
    quantile05 = df['gap'].quantile(0.05)
    quantile95 = df['gap'].quantile(0.95)
    return df[(df['gap'] >= quantile05) & (df['gap']<=quantile95)]

## 1. In AWS Virginia

In [5]:
compare_timestamp('./data/bloxroute-pair-reserve-cloud-virginia.json.xz', './data/fullnode-pair-reserve-bulk-virginia.json.xz').describe()

,hash_code,gap
count,1.259830e+07,1.259830e+07
mean,-2.015912e+14,-5.062799e+02
std,5.327161e+18,5.274324e+02
min,-9.223370e+18,-2.910000e+03
25%,-4.615148e+18,-6.940000e+02
50%,-8.629892e+14,-3.400000e+02
75%,4.616549e+18,-1.440000e+02
max,9.223372e+18,1.020000e+02


In [6]:
compare_timestamp('./data/fullnode-pair-reserve-bulk-virginia.json.xz', './data/fullnode-pair-reserve-bulk-header-virginia.json.xz').describe()

,hash_code,gap
count,1.392490e+07,1.392490e+07
mean,1.387331e+14,-2.882583e+01
std,5.325880e+18,2.225780e+01
min,-9.223370e+18,-1.180000e+02
25%,-4.612299e+18,-3.900000e+01
50%,-9.664096e+14,-2.600000e+01
75%,4.615081e+18,-1.300000e+01
max,9.223371e+18,2.000000e+00


In [7]:
compare_timestamp('./data/fullnode-pair-reserve-bulk-header-virginia.json.xz', './data/bloxroute-pair-reserve-gateway-virginia.json.xz').describe()

,hash_code,gap
count,1.377182e+07,1.377182e+07
mean,3.878259e+14,-3.029824e+01
std,5.326310e+18,1.080003e+01
min,-9.223370e+18,-7.000000e+01
25%,-4.612606e+18,-3.500000e+01
50%,-8.515627e+14,-2.800000e+01
75%,4.616401e+18,-2.300000e+01
max,9.223372e+18,-1.300000e+01


In [8]:
compare_timestamp('./data/bloxroute-pair-reserve-gateway-virginia.json.xz', './data/fullnode-pair-reserve-virginia.json.xz').describe()

,hash_code,gap
count,1.366082e+07,1.366082e+07
mean,3.378304e+14,-7.773835e+01
std,5.326179e+18,1.069626e+02
min,-9.223370e+18,-4.460000e+02
25%,-4.612675e+18,-1.270000e+02
50%,-1.277973e+15,-6.400000e+01
75%,4.616157e+18,1.000000e+00
max,9.223372e+18,7.800000e+01


**Conclusion: v5 -> 340ms -> v2 -> 26ms -> v3 -> 28ms -> v4 -> 64ms -> v1**

## 2. In Fremont IDC

In [9]:
compare_timestamp('./data/fullnode-pair-reserve-bulk-fremont.json.xz', './data/fullnode-pair-reserve-bulk-header-fremont.json.xz').describe()

,hash_code,gap
count,1.392475e+07,1.392475e+07
mean,-6.281547e+14,-3.114541e+01
std,5.326015e+18,1.827055e+01
min,-9.223370e+18,-6.800000e+01
25%,-4.613468e+18,-4.600000e+01
50%,-1.419434e+15,-3.100000e+01
75%,4.613636e+18,-1.600000e+01
max,9.223372e+18,2.000000e+00


In [10]:
compare_timestamp('./data/fullnode-pair-reserve-bulk-header-fremont.json.xz', './data/bloxroute-pair-reserve-gateway-fremont.json.xz').describe()

,hash_code,gap
count,1.373207e+07,1.373207e+07
mean,-2.447072e+14,-4.879597e+01
std,5.326245e+18,1.030387e+01
min,-9.223370e+18,-7.500000e+01
25%,-4.613351e+18,-5.600000e+01
50%,-1.288970e+15,-4.800000e+01
75%,4.615044e+18,-4.100000e+01
max,9.223372e+18,-2.700000e+01


In [11]:
compare_timestamp('./data/bloxroute-pair-reserve-gateway-fremont.json.xz', './data/fullnode-pair-reserve-fremont.json.xz').describe()

,hash_code,gap
count,1.360126e+07,1.360126e+07
mean,6.402046e+14,-7.692317e+01
std,5.326268e+18,1.078290e+02
min,-9.223370e+18,-2.710000e+02
25%,-4.612630e+18,-1.700000e+02
50%,-2.933300e+14,-7.700000e+01
75%,4.615803e+18,1.700000e+01
max,9.223372e+18,1.100000e+02


In [12]:
compare_timestamp('./data/fullnode-pair-reserve-fremont.json.xz', './data/bloxroute-pair-reserve-cloud-fremont.json.xz').describe()

,hash_code,gap
count,1.273854e+07,1.273854e+07
mean,-6.334931e+14,-7.257041e+01
std,5.326492e+18,2.023565e+02
min,-9.223370e+18,-7.180000e+02
25%,-4.614902e+18,-1.980000e+02
50%,-1.732694e+15,-7.200000e+01
75%,4.614830e+18,5.400000e+01
max,9.223372e+18,5.450000e+02


**Conclusion: v2 -> 31ms -> v3 -> 48ms -> v4 -> 77ms -> v1 -> 72ms -> cloud**

## 3. Compare Top1 between AWS Virginia and Fremont IDC

In [13]:
compare_timestamp('./data/bloxroute-pair-reserve-cloud-virginia.json.xz', './data/fullnode-pair-reserve-bulk-fremont.json.xz').describe()

,hash_code,gap
count,1.259740e+07,1.259740e+07
mean,-3.930686e+14,-1.321448e+01
std,5.326775e+18,1.457611e+02
min,-9.223370e+18,-6.710000e+02
25%,-4.615174e+18,-6.800000e+01
50%,-1.750918e+15,-2.000000e+00
75%,4.616081e+18,5.700000e+01
max,9.223372e+18,3.840000e+02


**Conclusion: v5 in AWS virginia is 2ms faster than v2 in Fremont IDC.**

## 4. Compare Two Fullnodes

In [18]:
compare_timestamp('./data/fullnode-pair-reserve-bulk-header-virginia.json.xz', './data/fullnode-pair-reserve-bulk-header-fremont.json.xz').describe()

,hash_code,gap
count,1.389367e+07,1.389367e+07
mean,-1.162133e+15,5.114367e+02
std,5.326132e+18,5.255902e+02
min,-9.223370e+18,-1.020000e+02
25%,-4.614049e+18,1.400000e+02
50%,-3.029049e+15,3.410000e+02
75%,4.613639e+18,7.140000e+02
max,9.223372e+18,2.819000e+03


## 5. Compare Two Gateways

In [19]:
compare_timestamp('./data/bloxroute-pair-reserve-gateway-virginia.json.xz', './data/bloxroute-pair-reserve-gateway-fremont.json.xz').describe()

,hash_code,gap
count,1.338754e+07,1.338754e+07
mean,-7.370417e+14,4.934824e+02
std,5.326324e+18,5.228973e+02
min,-9.223370e+18,-1.120000e+02
25%,-4.613762e+18,1.210000e+02
50%,-2.291492e+15,3.250000e+02
75%,4.614816e+18,6.990000e+02
max,9.223372e+18,2.776000e+03


## 6. Compare Cloud APIs between AWS Virginia and Fremont IDC

In [20]:
compare_timestamp('./data/bloxroute-pair-reserve-cloud-virginia.json.xz', './data/bloxroute-pair-reserve-cloud-fremont.json.xz').describe()

,hash_code,gap
count,1.163052e+07,1.163052e+07
mean,-4.095057e+14,-2.360176e+02
std,5.327213e+18,9.022455e+01
min,-9.223368e+18,-7.520000e+02
25%,-4.615862e+18,-2.510000e+02
50%,-1.262949e+15,-2.280000e+02
75%,4.616432e+18,-2.030000e+02
max,9.223372e+18,-3.700000e+01


**Conclusion: Accessing the cloud API from AWS Virginia is 297ms faster from Fremont, because the bloXroute websocket server `wss://virginia.bsc.blxrbdn.com/ws` is in AWS Virginia region**